In [2]:
import pandas as pd

In [3]:
df = pd.read_parquet(path='geo_data/tz_db/tz_dictionary.parquet')

# names belong to more populated countries take precedence.

In [19]:
exclude_exp = (r'[Rr]epublic|[Ss]tate|[Aa]uthority|Pradesh|'
               r'[Ww]est|[Ee]ast|[Ss]outh|[Nn]orth|[Rr]egion|'
               r'Assam|[Dd]ivision|[Nn]ation|[Qq]uarter|[Pp]rovince|'
               r'[Cc]ontinent|[Cc]ountry|[Dd]istrict|[Ii]sland')

temp = df.query('not name.str.contains(@exclude_exp) and not name==country and not name==continent')

country_ttl = temp.drop_duplicates(subset='name'). \
groupby('country', as_index=False)['population'].sum()

merged_with_sums = temp.merge(right=country_ttl, on='country')

res = merged_with_sums.sort_values(by=['population_y', 'population_x'], 
                                   ascending=False, ignore_index=True)
res['gmt_offset_minute'] = round(res['gmt_offset'] / 60).astype('int')
to_store = res.rename(columns=dict(population_x='population')).drop(columns='population_y')
to_store.to_parquet('geo_data/tz_sorted.parquet')